According to [this document](https://docs.sympy.org/latest/modules/utilities/lambdify.html), the module, `Lambdify`, 

> provides convenient functions to transform sympy expressions to lambda functions which can be used to calculate numerical values very fast.

Here is a study to investigate elapsed time of `Lambdify`.

In [ ]:
from IPython.display import display

from sympy import *
import numpy as np

Target functions are defined as follows:

In [ ]:
def f_target(x, a):
    """
    input:
        x: array(n, )
        a: array(4,)
    output:
        y: array(n,)
    , where y[i] = sum(a[j] * dx[i] **(j+1), j = 0...3), i = 0 ... n-1
    , dx[i] = x[i] - x[i-1], i = 1 ... n-1 
    and dx[0] = x[0] - x[n-1].
    """
    
    dx = x - np.roll(x,1) # (n,)
    X = np.stack((dx, dx**2, dx**3, dx**4), axis=-1) # (n,4)    
    y = X @ a # (n,)
    return y

Reimplement the above function in the *non-vectorized* way

In [ ]:
def f_target_nonvectorized(x, a):
    """
    input:
        x: array(n, )
        a: array(4,)
    output:
        y: array(n,)
    , where y[i] = sum(a[j] * dx[i] **(j+1), j = 0...3), i = 0 ... n-1
    , dx[i] = x[i] - x[i-1], i = 1 ... n-1 
    and dx[0] = x[0] - x[n-1].
    """
    
    n = x.shape[0]
    m = a.shape[0]
    y = np.zeros(n)
    for i in range(n):
        for j in range(m):
            y[i] += a[j] * (x[i] - x[(i-1)%n])**(j+1)
    return y

Set an array size `n` and a polinomial function degree `m` as follows:

In [ ]:
n = 2**5
m = 4

# Task 1:

Measure elapsed time for running `f_target` and its non-vectorized implementation, `f_target_vectorized`, as bentchmark:

In [ ]:
x = np.random.randn(n)
a = np.random.randn(m)
%timeit f_target(x, a)
# 38.8 µs ± 3.55 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
%timeit f_target_nonvectorized(x, a)
# 182 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Let define a symbolised expression of the target function, `f_target` and lambdify it:

In [ ]:
x = np.array(symbols("x:%d" % n))
a = np.array(symbols("a:%d" % m))
exprs = np.array([expand(expr) for expr in f_target(x, a)])

# display the expression of the first element as example:
display(exprs[0])

f_target_lambdified = lambdify((x,a), exprs, modules="numpy")

Run a sanitycheck:

In [ ]:
x = np.random.randn(n)
a = np.random.randn(m)
y = f_target_lambdified(x, a)
yTrue = f_target(x,a)

assert np.all(np.isclose(yTrue, y))

Measure elapsed time for running the *lamdified*  target function, `f_target_lambdified` to compare with the benchmark:

In [ ]:
x = np.random.randn(n)
a = np.random.randn(m)
%timeit f_target_lambdified(x, a)
# 333 µs ± 50.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Findings:
* Elapsed time measured in my environment shows that the lambdified `f_target` is slower than a vectorized one 
* and the time is doubled by the time of the non-vectorized implementation, `f_target_nonvectorized`.

# Task 2: how about apply a redefinition of lambdified target function?

Investigate the case where symbolised expressions of the target function are *NOT* expanded:

In [ ]:
x = np.array(symbols("x:%d" % n))
a = np.array(symbols("a:%d" % m))
exprs = np.array([expr for expr in f_target(x, a)])

# display the expression of the first element as example:
display(exprs[0])

f_target_lambdified = lambdify((x,a), exprs, modules="numpy")

Measure elapsed time for running the *redefined* implementation:

In [ ]:
x = np.random.randn(n)
a = np.random.randn(m)
%timeit f_target_lambdified(x, a)
# 91.6 µs ± 3.95 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Findings:
* The elapsed time is still slower than the time of the vectorized implementation, `f_target`,
* though, it's faster than the non-vectorized one,`f_target_vectorized`

# Task 3: 
measure elapsed time for getting the jacobian of the target function:

In [ ]:
x = np.array(symbols("x:%d" % n))
a = np.array(symbols("a:%d" % m))
jacobian_exprs = np.array([expr.diff(x[i]) for expr in f_target(x, a) for i in range(n)]).reshape(n,n)

# display the expression of the first element as example:
display(jacobian_exprs[0,0])

jacobian_lambdified = lambdify((x,a), jacobian_exprs, modules="numpy")

Measure elapsed time for running a process of the calculation of jacobian, `jacobian_lambdified`:

In [ ]:
x = np.random.randn(n)
a = np.random.randn(m)
%timeit jacobian_lambdified(x, a)
# 230 µs ± 29.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Findings:
* Calculating a jacobian of the target function can take a bit longer doubled time than a single run of the (lambdified) target function